Added wait time for the api response instead of arbitrary wait time. (Based on jobID api call thingy)
Made Dataframe to store the Suprise status along with text and timeOffset

In [1]:
from cgitb import text
import requests
import time
import os
import pprint
baseUrl = "https://api-labs.symbl.ai/v1/conversations/{conversationId}/messages"
# Generated using Submit text end point
from get import conversationId
from get import jobId


url = baseUrl.format(conversationId=conversationId)

# set your access token here. See https://docs.symbl.ai/docs/developer-tools/authentication
access_token = os.getenv('access_token')
access_token = str(access_token)


headers = {
    'Authorization': 'Bearer ' + access_token,
    'Content-Type': 'application/json'
}

params = {
    'verbose': True,  # <Optional, boolean| Gives you word level timestamps of each sentence.>
    'sentiment': True  # <Optional, boolean| Give you sentiment analysis on each message.>
}

responses = {
    401: 'Unauthorized. Please generate a new access token.',
    404: 'The conversation and/or it\'s metadata you asked could not be found, please check the input provided',
    500: 'Something went wrong! Please contact support@symbl.ai'
}

jobUrl = "https://api-labs.symbl.ai/v1/job/{jobId}"
finalUrl = jobUrl.format(jobId=jobId)


#Check the jobID api to see if the job is finished
while requests.request("GET", finalUrl, headers=headers).json()['status'] == 'in_progress':
    #if in progress, wait for 5 second
    print('Waiting for API to finish...')
    time.sleep(5)

print('')
print('Got Response!')


response = requests.request("GET", url, headers=headers)

conversationId => 6165254194069504
jobId => 566fffef-ca95-4b0a-8f1d-1893fa6539ca
Waiting for API to finish...
Waiting for API to finish...
Waiting for API to finish...
Waiting for API to finish...
Waiting for API to finish...
Waiting for API to finish...
Waiting for API to finish...
Waiting for API to finish...
Got Response!



In [2]:
if response.status_code == 200:
    # Successful API execution
    # messages is a list of id, text, from, startTime, endTime, conversationId, words, phrases, sentiment
    #print("messages => " + str(response.json()['messages']))
    for i in range (len(response.json()['messages'])):
        a = response.json()['messages'][i]['text']
        b = response.json()['messages'][i]['timeOffset']
        print (a,"at ",b )

elif response.status_code in responses.keys():
    print(responses[response.status_code])  # Expected error occurred
else:
    print("Unexpected error occurred. Please contact support@symbl.ai" + ", Debug Message => " + str(response.text))

It's cheaper that way might break here for Mason MI. at  5
Yeah, he's buried it. at  7.7
What a start for Chelsea, but he's had a stroke of luck here. at  10
But what a cool finish, absolutely brilliant. at  13.3
And now with the outswinger Toni, really what a big goal that is from the big German. at  21.3
Jeez, can't a look at Alonso space left-hand side, can take finally tease him up. at  33.9
He's come back for a living. at  38.6
This is absolutely extraordinary Marcus Alonzo. at  42.6
Does he keep them handball goal is disallowed? at  53.1
Goodness Me? at  57.2
Is that going to be the turning point of this game? at  60.2
That's hope not. at  62
But his men deep. at  62.8
And that is Benjamin, that's off the crossbar put things in and it's a free header to be honest and for him, I was expecting more. at  65.9
Rudiger with the goals that have got his it to 33 on aggregate. at  78
He is Vernon looking to get Chelsea serve at night. at  81.3
Timo Bernhard and still he goes away. at  83

In [3]:
import pandas as pd

In [4]:
text = []
timeOffset = []

for i in range (len(response.json()['messages'])):
    a = response.json()['messages'][i]['text']
    b = response.json()['messages'][i]['timeOffset']
    text.append(a)
    timeOffset.append(b)

In [5]:
sup = []
import text2emotion as te

for i in text:
    emote = te.get_emotion(i)
    sup.append(emote['Surprise'])

[nltk_data] Downloading package stopwords to /home/venom/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/venom/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/venom/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [6]:
df = pd.DataFrame({'text':text, 'timeOffset':timeOffset, 'Surprise':sup})

In [7]:
df

,text,timeOffset,Surprise
0,It's cheaper that way might break here for Mas...,5.0,0.50
1,"Yeah, he's buried it.",7.7,0.00
2,"What a start for Chelsea, but he's had a strok...",10.0,0.00
3,"But what a cool finish, absolutely brilliant.",13.3,0.00
4,"And now with the outswinger Toni, really what ...",21.3,1.00
5,"Jeez, can't a look at Alonso space left-hand s...",33.9,0.00
6,He's come back for a living.,38.6,0.00
7,This is absolutely extraordinary Marcus Alonzo.,42.6,0.00
8,Does he keep them handball goal is disallowed?,53.1,0.00
9,Goodness Me?,57.2,0.00


In [2]:
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

#Takes some time to load the model

In [15]:
#First time processing takes around 5 seconds, subsequent times take less (less than a second)

sentences = ["What an six!", "That was an excellent six by Rohit Sharma"]
#Compute embedding for both lists
embedding_1= model.encode(sentences[0])
embedding_2 = model.encode(sentences[1])

output = util.pytorch_cos_sim(embedding_1, embedding_2)

print(output.tolist()[0][0])

0.5081178545951843


PCA on audio

In [1]:
#import pca libs
import numpy as np
import matplotlib.pyplot as plt
#sk learn PCA
from sklearn.decomposition import PCA

In [2]:
from IPython.display import Audio
from scipy.io import wavfile

samplerate, audiotest1 = wavfile.read('audiotest1.wav')

start = samplerate * 14 # 10 seconds in
end = start + samplerate * 10 # 5 second duration
Audio(data=audiotest1[start:end, 0], rate=samplerate)

In [10]:
from sklearn.decomposition import PCA
import numpy as np

def pca_reduce(signal, n_components, block_size=1024):
    
    # First, zero-pad the signal so that it is divisible by the block_size
    samples = len(signal)
    hanging = block_size - np.mod(samples, block_size)
    padded = np.lib.pad(signal, (0, hanging), 'constant', constant_values=0)
    
    # Reshape the signal to have 1024 dimensions
    reshaped = padded.reshape((len(padded) // block_size, block_size))
    
    # Second, do the actual PCA process
    pca = PCA(n_components=n_components)
    pca.fit(reshaped)
    
    transformed = pca.transform(reshaped)
    reconstructed = pca.inverse_transform(transformed).reshape((len(padded)))
    print(sum(pca.explained_variance_ratio_))
    return pca, transformed, reconstructed

In [12]:
tabulasa_left = audiotest1[:,0]

_, _, reconstructed = pca_reduce(tabulasa_left, 256, 4096)

Audio(data=reconstructed[start:end], rate=samplerate)

0.9073385695045846
